# Predicting House Price Class

<hr>

## Goal of the Project
- The real estate is not so interested in finding what matters most to find house price
- But interested in which range a house is in
- There are 3 classes: 33% cheapest, 33% mid-range, 33% expensive houses.
- He needs to find which parameters matter most to determine that